# Point to grid conversion

This code takes the georeferenced points and related them to a fishnet
grid of 1km x 1km scale.

This will be for both for Australian Plague Locust and Desert Locust

In [5]:
import pandas as pd
import geopandas as gpd
import os

In [7]:
os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
os.getcwd()

'/home/datascience/cleaned_aplc_soils_project'

In [8]:
# This function takes the point dataset and creates a sparse fishnet grid that overlaps with all points. 
# there is a user species cell size and few other handy parameters
# it defaults to parallel processing but can be turned off

# it creates a bounding box, creates a coarse grid, filters out unused grids and then within each coarse grid, it creates the fine scale grid
# at the speicifed cell size and then removes unused grids.

with open('../cleaned_aplc_soils_project/scripts/functions/fishnet_grid_function.py') as f:
    code = f.read()

# Execute the code
exec(code)


## Australian plague locust data

This dataset is comprised of several species but the records are biased
towards the more economically important species (e.g. australian plauge
locust)

## Species identifier

The `Species` column is the identifier per species

-   10 = no locust/grasshopper found
-   11 = Australian plague locust (*Chortocietes terminifera*)
-   12 = Spur-throated locust (*Austracris guttulosa*)
-   15 = Small plague locust (*Austroicetes cruciata*)
-   19 = Eastern plague grasshopper (*Oedaleus australis*)

### Species data management

For each species, we need to combined the species specific ID with the
`10` observations (nil-observations) and observations in the other
species categories that are nil. This will provide us with all the
records that a specific grasshopper species was present and combine it
with all the nil observation possible.

In [32]:
df = pd.read_csv('../cleaned_aplc_soils_project/data/raw/survey_data/APLC_surveys_2000-2017.csv')

In [34]:
import pandas as pd

def filter_and_update_species(df, species_list, target_species):
    and_nil = df['Species'].isin(species_list)
    all_other_nil = (
        ~df['Species'].isin(species_list) &
        (df['Nymph Density'] == 0) &
        (df['Adult Density'] == 0)
    )
    filtered_df = df[and_nil | all_other_nil].copy()  # Use copy() to avoid SettingWithCopyWarning
    filtered_df['Species'] = target_species
    return filtered_df

# Define the list of target species and corresponding species lists
species_combinations = [
    ([10, 11], 11),
    ([10, 12], 12),
    ([10, 15], 15),
    ([10, 19], 19)
]

# Use a list comprehension to generate all DataFrames in one step
full_species_data = pd.concat(
    [filter_and_update_species(df, species_list, target_species) 
     for species_list, target_species in species_combinations], 
    ignore_index=True
).drop_duplicates()  # Optionally drop duplicates

# final_df now contains the combined result
full_species_data.head()

## Fishnet grid construction

I first create the sparse fishnet grid (roughly 1km x 1km) and then
aggregate the species data to the polygons. This is done BY species and
not between.

In [37]:

aplc_survey_dataframe = create_grids_parallel(
        full_species_data,
        longitude_column='Longitude',
        latitude_column='Latitude',
        csv_crs='EPSG:4326',
        grid_crs='EPSG:4326',
        transformation_crs='EPSG:4326',
        final_cell_size=0.01,
        coarse_cell_size=None,
        parallel=True
    )



Processing Polygons: 100%|██████████| 1315/1315 [00:14<00:00, 88.39it/s] 

In [38]:
# Add an index to the polygons
## this will allow us to join tables together (and make GEE easier)

aplc_survey_dataframe = gpd.GeoDataFrame(aplc_survey_dataframe)
aplc_survey_dataframe['polygon_id'] = aplc_survey_dataframe.index
aplc_survey_dataframe.set_crs(epsg=4326, inplace=True)

67176 rows × 2 columns

### Lets summarize the APL point data to the polygon grid data

In [42]:
columns_to_keep = ['Longitude', 'Latitude', 'Date','Species','Data Quality','Nymph Density','Adult Density']
APL_dat = final_df[columns_to_keep]


In [43]:
# Convert DataFrame to GeoDataFrame

gdf_points = gpd.GeoDataFrame(
    APL_dat,
    geometry=gpd.points_from_xy(APL_dat.Longitude, APL_dat.Latitude)
)

gdf_points.set_crs(epsg=4326, inplace=True)


309640 rows × 8 columns

In [44]:
joined_gdf = gpd.sjoin(gdf_points, aplc_survey_dataframe, how="left", predicate='within')


In [45]:
# I wanted to ensure every point has an assoicated polygon_id 

## which they do

na_counts = joined_gdf.isna().sum()

print(na_counts)

Longitude        0
Latitude         0
Date             0
Species          0
Data Quality     0
Nymph Density    0
Adult Density    0
geometry         0
index_right      0
polygon_id       0
dtype: int64

In [46]:

# Custom aggregation function to get min, median, and max dates
def date_aggregations(series):
    series = pd.to_datetime(series, format='%d-%b-%y') # converts to a datatime
    return [series.min(), series.median(), series.max()]

# aggregation function to count unique values
def count_unique_values(series):
    return series.value_counts().to_dict()

# Group by polygon geometry and aggregate the data
aggregated_gdf = joined_gdf.groupby(['Species', 'polygon_id']).agg({
    'Date': date_aggregations,
    'Longitude': 'median',
    'Latitude': 'median',
    'Data Quality': count_unique_values,
    'Nymph Density': count_unique_values,
    'Adult Density': count_unique_values
}).reset_index()


In [47]:
aggregated_gdf

177613 rows × 8 columns

In [48]:
# Lets count up the total number of observations in the dataframe:

# Define the aggregation functions for each column
agg_funcs = {
    'Data Quality': 'count',  
    'Adult Density': 'count',    
    'Nymph Density': 'count'    
}

# Perform groupby operation and aggregate
grouped_joined_gdf = joined_gdf.groupby(['Species', 'polygon_id']).agg(agg_funcs).reset_index()

# Rename the aggregated columns
grouped_joined_gdf.rename(columns={
    'Data Quality': 'Data Quality Total Count',
    'Adult Density': 'Adult Density Total Count',
    'Nymph Density': 'Nymph Density Total Count'
}, inplace=True)


# These three columns should agree with one another....
## if any rows dont -- this command filters for them
grouped_joined_gdf[
    (grouped_joined_gdf['Data Quality Total Count'] != grouped_joined_gdf['Adult Density Total Count']) |
    (grouped_joined_gdf['Data Quality Total Count'] != grouped_joined_gdf['Nymph Density Total Count']) |
    (grouped_joined_gdf['Adult Density Total Count'] != grouped_joined_gdf['Nymph Density Total Count'])
]

In [49]:
final_df = pd.merge(aggregated_gdf, grouped_joined_gdf, on=['Species', 'polygon_id'], how='left')
final_df = pd.merge(final_df, aplc_survey_dataframe, on='polygon_id', how='left')
final_df.head()
na_counts = final_df.isna().sum()

print(na_counts)

Species                      0
polygon_id                   0
Date                         0
Longitude                    0
Latitude                     0
Data Quality                 0
Nymph Density                0
Adult Density                0
Data Quality Total Count     0
Adult Density Total Count    0
Nymph Density Total Count    0
geometry                     0
dtype: int64

### The APLC data is complete – lets write to disk

In [51]:
final_df.to_csv('../cleaned_aplc_soils_project/data/processed/spatial_modeling/aplc_data_aggregated_to_polygon_grid.csv')

## Desert Locust dataset

now there are two public desert locust datasets: ‘hoppers’ and ‘bands’

I’m not going to do too much with this dataset – I just want to get the
polygon grids constructed and potentially return to it if we need to

In [3]:
bands_df = pd.read_csv('cleaned_aplc_soils_project/data/raw/survey_data/desert_locust/Bands_Public_3599395965147926909.csv')
hoppers_df = pd.read_csv('cleaned_aplc_soils_project/data/raw/survey_data/desert_locust/Hoppers_Public_7481916456258955251.csv')
swarms_df = pd.read_csv('cleaned_aplc_soils_project/data/raw/survey_data/desert_locust/Swarms_Public_891175385870988899.csv')
adults_df = pd.read_csv('cleaned_aplc_soils_project/data/raw/survey_data/desert_locust/Adults_Public_-3113927055772371667.csv')


columns_to_keep = ['STARTDATE', 'LOCPRESENT', 'LOCRELIAB','REPRELIAB', 'SHPDENISOL', 'SHPDENSCAT', 
                       'SHPDENGRP', 'SHPDENUNK', 'GHPDENLOW', 'GHPDENMED', 'GHPDENHI', 
                       'GHPDENUNK', 'SADDENISOL', 'SADDENSCAT', 'SADDENGRP', 'SADDENUNK',
                       'GADDENLOW', 'GADDENMED', 'GADDENHI', 'GADDENUNK', 'CAT', 'x', 'y']
# Check if columns in all DataFrames match

# Check if columns in all DataFrames match
if bands_df.columns.equals(hoppers_df.columns) and \
   bands_df.columns.equals(swarms_df.columns) and \
   bands_df.columns.equals(adults_df.columns):
    
    # Columns match, concatenate the DataFrames
    combined_dl_df = pd.concat([bands_df, hoppers_df, swarms_df, adults_df], ignore_index=True)
    
    # Filter for specified columns
    combined_dl_df = combined_dl_df[columns_to_keep]
else:
    # Columns don't match, raise an error
    raise ValueError("Columns of the DataFrames do not match.")

In [4]:
combined_dl_df

250705 rows × 23 columns

In [5]:
# Map specific columns to desired values
column_mapping = {
    'LOCPRESENT': {1: 'present', 2: 'absent'},
    'REPRELIAB': {0: 'dubious', 1: 'reliable'},
    'SHPDENISOL': {0: 'no', 1: 'isolated'},
    'SHPDENSCAT': {0: 'no', 1: 'scattered'},
    'SHPDENGRP': {0: 'no', 1: 'group'},
    'SHPDENUNK': {0: 'no', 1: 'unkown'},
    'GHPDENLOW': {0: 'no', 1: 'low'},
    'GHPDENMED': {0: 'no', 1: 'medium'},
    'GHPDENHI': {0: 'no', 1: 'high'},
    'GHPDENUNK': {0: 'no', 1: 'unkown'},
    'SADDENISOL': {0: 'no', 1: 'isolated'},
    'SADDENSCAT': {0: 'no', 1: 'scattered'},
    'SADDENGRP': {0: 'no', 1: 'group'},
    'SADDENUNK': {0: 'no', 1: 'unkown'},
    'GADDENLOW': {0: 'no', 1: 'low'},
    'GADDENMED': {0: 'no', 1: 'medium'},
    'GADDENHI': {0: 'no', 1: 'high'},
    'GADDENUNK': {0: 'no', 1: 'unknown'}
}

for column, mapping in column_mapping.items():
    combined_dl_df[column] = combined_dl_df[column].map(mapping)

In [6]:
# Assuming combined_dl_df is your DataFrame containing the columns

# Define column groups
column_groups = {
    'SHP': ['SHPDENISOL', 'SHPDENSCAT', 'SHPDENGRP', 'SHPDENUNK'],
    'GHP': ['GHPDENLOW', 'GHPDENMED', 'GHPDENHI', 'GHPDENUNK'],
    'SAD': ['SADDENISOL', 'SADDENSCAT', 'SADDENGRP', 'SADDENUNK'],
    'GAD': ['GADDENLOW', 'GADDENMED', 'GADDENHI', 'GADDENUNK']
}

# Create a new DataFrame to store collapsed values
collapsed_df = pd.DataFrame()

# Iterate over each group
for group_prefix, columns in column_groups.items():
    # Select columns belonging to the current group
    group_data = combined_dl_df[columns]
    # Determine collapsed value for each row in the group
    collapsed_value = group_data.apply(lambda row: '_'.join(row[row != 'no']) if sum(row != 'no') > 1 else row[row != 'no'].values[0] if sum(row != 'no') == 1 else 'no', axis=1)
    # Assign collapsed value to a new column in the collapsed_df
    collapsed_df[group_prefix] = collapsed_value

# Drop the original columns that were collapsed from the original DataFrame
collapsed_columns = [column for columns in column_groups.values() for column in columns]
original_columns = [column for column in combined_dl_df.columns if column not in collapsed_columns]
original_df = combined_dl_df[original_columns]

# Concatenate the original DataFrame with the collapsed DataFrame
final_df = pd.concat([original_df, collapsed_df], axis=1)

# Display the final DataFrame
final_df

250705 rows × 11 columns

So there is a lot of hand-waviness when it comes to this data.
`LOCPRESENT` does not necessarily agree with the `SHP`, `GHP`, `SAD`, or
`GAD` columns.

I think for I will take only into consideration the SHP, GHP, SAD, and
GAD columns and loosely categorize them into broad catorgies of dense
vs. not.

With this said, there are a bunch of overlaps….check out the counts
table below.

In [7]:
final_df.groupby(['SHP','GHP']).count()['x']

SHP                       GHP            
group                     high                 3160
                          low                   600
                          low_high               76
                          low_medium             38
                          low_medium_high         8
                          medium               3863
                          medium_high           395
                          no                  35076
                          unkown                113
isolated                  high                    9
                          low                    44
                          medium                 26
                          no                   8257
isolated_group            medium                  9
                          no                      3
isolated_scattered        no                     46
isolated_scattered_group  no                      2
no                        high                 7613
                      

I think we can debate for a while the correct way to handle this data.

For now, lets just go forward and do the polygon gridding like we did
with the APLC data.

In [8]:

dl_survey_dataframe = create_grids_parallel(
        final_df,
        longitude_column='x',
        latitude_column='y',
        csv_crs='EPSG:4326',
        grid_crs='EPSG:4326',
        transformation_crs='EPSG:4326',
        final_cell_size=0.01,
        coarse_cell_size=None,
        parallel=True
    )

Processing Polygons: 100%|██████████| 287/287 [02:21<00:00,  2.02it/s]

In [9]:
# Add an index to the polygons
## this will allow us to join tables together (and make GEE easier)

dl_survey_dataframe = gpd.GeoDataFrame(dl_survey_dataframe)
dl_survey_dataframe['polygon_id'] = dl_survey_dataframe.index
dl_survey_dataframe.set_crs(epsg=4326, inplace=True)

130690 rows × 2 columns

In [10]:
# Convert DataFrame to GeoDataFrame

gdf_points = gpd.GeoDataFrame(
    final_df,
    geometry=gpd.points_from_xy(final_df.x, final_df.y)
)

gdf_points.set_crs(epsg=4326, inplace=True)


250705 rows × 12 columns

In [11]:
joined_gdf = gpd.sjoin(gdf_points, dl_survey_dataframe, how="left", predicate='within')


In [12]:
joined_gdf

251566 rows × 14 columns

In [15]:

# Custom aggregation function to get min, median, and max dates
def date_aggregations(series):
    series = pd.to_datetime(series, format='%m/%d/%Y %I:%M:%S %p') # converts to a datatime
    return [series.min(), series.median(), series.max()]

# aggregation function to count unique values
def count_unique_values(series):
    return series.value_counts().to_dict()

# Group by polygon geometry and aggregate the data
aggregated_gdf = joined_gdf.groupby(['polygon_id']).agg({
    'STARTDATE': date_aggregations,
    'x': 'median',
    'y': 'median',
    'LOCPRESENT': count_unique_values,
    'REPRELIAB': count_unique_values,
    'CAT': count_unique_values,
    'SHP': count_unique_values,
    'GHP': count_unique_values,
    'SAD': count_unique_values,
    'GAD': count_unique_values
}).reset_index()


In [23]:
# Lets count up the total number of observations in the dataframe:

# Define the aggregation functions for each column
agg_funcs = {
    'CAT': 'count',
    'SHP': 'count',
    'GHP': 'count',
    'SAD': 'count',
    'GAD': 'count',
    'LOCPRESENT': 'count',    
    'REPRELIAB': 'count'    
}

# Perform groupby operation and aggregate
grouped_joined_gdf = joined_gdf.groupby(['polygon_id']).agg(agg_funcs).reset_index()

# Rename the aggregated columns
grouped_joined_gdf.rename(columns={
    'CAT': 'total_CAT_count',
    'SHP': 'total_SHP_count',
    'GHP': 'total_GHP_count',
    'SAD': 'total_SAD_count',
    'GAD': 'total_GAD_count',
    'LOCPRESENT': 'total_LOCPRESENT_count',    
    'REPRELIAB': 'total_REPRELIAB_count'    
}, inplace=True)


# These three columns should agree with one another....
## if any rows dont -- this command filters for them

grouped_joined_gdf['unique_counts'] = grouped_joined_gdf.apply(lambda row: len(set(row[1:])), axis=1)
inconsistent_rows = grouped_joined_gdf[grouped_joined_gdf['unique_counts'] > 1].drop(columns=['unique_counts'])



In [24]:
inconsistent_rows

540 rows × 8 columns

So the reliability column does agree – but thats it. I dont think its a
big concern…

In [29]:
final_df = pd.merge(aggregated_gdf, grouped_joined_gdf, on=['polygon_id'], how='left')
final_df = pd.merge(final_df, dl_survey_dataframe, on='polygon_id', how='left')
final_df


130690 rows × 20 columns

In [28]:
na_counts = final_df.isna().sum()

print(na_counts)

polygon_id                0
STARTDATE                 0
x                         0
y                         0
LOCPRESENT                0
REPRELIAB                 0
CAT                       0
SHP                       0
GHP                       0
SAD                       0
GAD                       0
total_CAT_count           0
total_SHP_count           0
total_GHP_count           0
total_SAD_count           0
total_GAD_count           0
total_LOCPRESENT_count    0
total_REPRELIAB_count     0
unique_counts             0
geometry                  0
dtype: int64

In [30]:
final_df.to_csv('cleaned_aplc_soils_project/data/processed/spatial_modeling//dl_data_aggregated_to_polygon_grid.csv')